<a href="https://colab.research.google.com/github/Awesome-Areej/improved-doodle/blob/main/Jan_25_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

I have coded the AI depicted in my advertisement. I was able to do so by using Python, Google Cloud Vision, and Text Generation Application Programming Interfaces (API). The first thing I did was collect as much dialogue as I could from Animal Crossing using screenshots and videos. The text found in the aforementioned photos was retrieved by utilizing Google API. After compiling the retrieved text, I trained a recurrent neural network (RNN) on these dialogues. By doing this, the RNN was taught how to generate original text from what it had been trained on earlier.

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf
import numpy as np
import os
import time

### Read the data

First, look in the text:

In [3]:
# # Read, then decode for py2 compat.
# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# # length of text is the number of characters in it
# print(f'Length of text: {len(text)} characters')

In [4]:
from google.colab import files
uploaded = files.upload()


Saving merged.txt to merged.txt


In [5]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "merged.txt" with length 91530 bytes


In [6]:
text = uploaded['merged.txt']
text = text.decode(encoding='utf-8')
print(text[:250])


Sherb
You look warm, champ.
As warm as a cinnamon roll fresh
from the oven.

Ankha
If it had cheeks, I would pinch them!

Sherb
NOTHING MAKES SENSE IN
THE DARK!

Molly
Always good to see your smile again,
darling.

Вeau
So many folks here to see ME.


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

86 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[47, 48, 49, 50, 51, 52, 53], [70, 71, 72]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [12]:

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [13]:

tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(91366,), dtype=int64, numpy=array([ 1, 39, 54, ..., 10,  1,  1])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))



S
h
e
r
b


Y
o
u


In [18]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b'S' b'h' b'e' b'r' b'b' b'\n' b'Y' b'o' b'u' b' ' b'l' b'o' b'o'
 b'k' b' ' b'w' b'a' b'r' b'm' b',' b' ' b'c' b'h' b'a' b'm' b'p' b'.'
 b'\n' b'A' b's' b' ' b'w' b'a' b'r' b'm' b' ' b'a' b's' b' ' b'a' b' '
 b'c' b'i' b'n' b'n' b'a' b'm' b'o' b'n' b' ' b'r' b'o' b'l' b'l' b' '
 b'f' b'r' b'e' b's' b'h' b'\n' b'f' b'r' b'o' b'm' b' ' b't' b'h' b'e'
 b' ' b'o' b'v' b'e' b'n' b'.' b'\n' b'\n' b'A' b'n' b'k' b'h' b'a' b'\n'
 b'I' b'f' b' ' b'i' b't' b' ' b'h' b'a' b'd' b' ' b'c' b'h' b'e' b'e'
 b'k' b's' b','], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\nSherb\nYou look warm, champ.\nAs warm as a cinnamon roll fresh\nfrom the oven.\n\nAnkha\nIf it had cheeks,'
b' I would pinch them!\n\nSherb\nNOTHING MAKES SENSE IN\nTHE DARK!\n\nMolly\nAlways good to see your smile aga'
b'in,\ndarling.\n\n\xd0\x92eau\nSo many folks here to see ME.\nWow. I feel like Mr. Popular,\na huh huh huh, saltlic'
b"k.\n\n\nRedd\nLet me tell ya, selling art is a\ntough business.\n\nSherb\nHappyyy New Year! It's 2022 startin"
b"g\ntoday! That's pretty cool!\n\nMarina\nFor me, it'd have to be out under\na clear sky, full of stars...a"


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\nSherb\nYou look warm, champ.\nAs warm as a cinnamon roll fresh\nfrom the oven.\n\nAnkha\nIf it had cheeks'
Target: b'Sherb\nYou look warm, champ.\nAs warm as a cinnamon roll fresh\nfrom the oven.\n\nAnkha\nIf it had cheeks,'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [26]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 87) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  22272     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  89175     
                                                                 
Total params: 4,049,751
Trainable params: 4,049,751
Non-trainable params: 0
_________________________________________________________________


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [32]:
sampled_indices

array([ 1, 46, 55, 26, 84, 44, 49, 25, 19, 22, 34, 85, 75,  1,  3, 15, 29,
       57, 18, 16, 17, 52, 14, 75, 81,  8, 50, 35, 13,  2, 61,  3,  9, 77,
       18, 19, 47,  2, 86, 69, 47, 82, 26, 56, 22, 67,  1,  5, 73, 17, 16,
       41, 40, 52,  6, 30, 66, 40, 29, 27, 52, 26, 78, 59, 22, 74, 11, 82,
       45, 32, 40,  5, 37, 39, 40, 36, 64,  6, 63, 68, 46, 57, 54,  4, 41,
       80, 47, 64, 75, 81, 71, 28, 55, 67, 26, 80, 84, 25, 55, 27])

Decode these to see the text predicted by this untrained model:

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"Tangy\nI'm SO going to show this off in my\nhouse! As sooooon as I figure\nout where to put it!\n\n\nTangy"

Next Char Predictions:
 b'\nZiE\xe1\xba\xa1XcD8AM\xe2\x80\x93\xc3\xa8\n!4Hk756f3\xc3\xa8\xd0\x9d,dN2 o!-\xc8\x9b78a \xe2\x80\xa2wa\xd0\xb0EjAu\n&\xc3\x8965UTf\'ItTHFfE\xd0\x90mA\xc3\xa10\xd0\xb0YKT&PSTOr\'qvZkh"U\xd0\x9aar\xc3\xa8\xd0\x9dyGiuE\xd0\x9a\xe1\xba\xa1DiF'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 87)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.4659896


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [36]:
tf.exp(mean_loss).numpy()

87.00709

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [37]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [38]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [39]:
EPOCHS = 40

In [40]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
14/14 [==============================] - 7s 174ms/step - loss: 4.4261
Epoch 2/40
14/14 [==============================] - 3s 145ms/step - loss: 3.5829
Epoch 3/40
14/14 [==============================] - 2s 136ms/step - loss: 3.2067
Epoch 4/40
14/14 [==============================] - 2s 133ms/step - loss: 2.9867
Epoch 5/40
14/14 [==============================] - 2s 134ms/step - loss: 2.7572
Epoch 6/40
14/14 [==============================] - 2s 137ms/step - loss: 2.6246
Epoch 7/40
14/14 [==============================] - 2s 135ms/step - loss: 2.5468
Epoch 8/40
14/14 [==============================] - 2s 134ms/step - loss: 2.4912
Epoch 9/40
14/14 [==============================] - 2s 136ms/step - loss: 2.4428
Epoch 10/40
14/14 [==============================] - 2s 138ms/step - loss: 2.3967
Epoch 11/40
14/14 [==============================] - 2s 134ms/step - loss: 2.3476
Epoch 12/40
14/14 [==============================] - 2s 134ms/step - loss: 2.2977
Epoch 13/40
14/14 [======

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [43]:
start = time.time()
states = None
next_char = tf.constant(['Sherb'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Sherb
That's just this spowar! Maybig!
I deach. It's like...

Molly
Para to be a soustedle pizza!

Isabelle
That's mury improsed. Wangioted
ever now meane a dream.

Candy
I boggers now, happy fibs, plays.

Resetti
Chere.
What do you think, ayup yup?

Winnie
Has it happen it so Flaud
So piffee by glow for mo
work! 's for a quick if
I at ite evry, if, Ploee.

Shep
You like to make so ranching to
me. June always love having.
I'm so preasy comer defion.


Tangy
I better don't spark it ond foring! But
and I just love
Oh! The sooke 's don't know!

Happy New Year! It's 202... reau!
"on't not jor Niso!
Hey! Happy Niko.
Happy New Year's fooks like shows.

Julian
Has it belong regit, ma!

Shep
Sartlicula have friendurl. Ever
gef them, are jake a nap!


Вeau
Do the 'll kinds of Feelint
relaving the look can ress all kinds
of U00 slick. My glae have
lessmetians in a ranncern.

Sherb
I do a gifu, have you been
tell you...
"gued to be a rongorywh.

Tangy
Bue backy out anot have
beften a areej about


The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [52]:
start = time.time()
states = None
next_char = tf.constant(['ISABELLE:', 'ISABELLE:', 'ISABELLE:', 'ISABELLE:', 'ISABELLE:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ISABELLE:\n\nMoWely\nI\'ve been hers becidee. Took-My Nico\nwidally sayling-comait in says..\n\nTangy\nYeAzA! HHOR!\n\nMolly\nPluave is see each of ispand is\nwert Tongy?\n\nZulkn\nDe dance pracince in thilw\nwere our hard day. That\'s\nwak on khen juam juats are\nastumely those youn tom\nthe wowing sheghay! You super molt\nsonethel cize. Whice im rinew!\n\n\nMolly\nI hus, that\'s my ted make fIVE brouht...\n\n\nKe\nBut Pas, a\njust Snow you,\nshilioincer? Or Hay yaul"\ngable mike always great\nspartlieving. Ohen it fall as\nI caul the new year with everybody!\n\nWartell\nJust becauge yue?\n\nTangy\nI can jo know. I just can\'t will be\nany one of togetfulls\nbutuse I\'m one of on the place\nin the propebsed. Bot you two\nare sometime out for\nme come out... Gulin\' movememe.\n\nReneigh\nLy ereej! I can\'t wait it belong\nup now. Then should me a\nREy scould bit hissile,\nand I do out for a biller about a\npeyching flower-sty onlony...\n\nAvarel\'\n\n\nAren\nSonsy-This is

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [45]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['Isabelle:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [50]:
model.fit(dataset, epochs=1)

14/14 [==============================] - 4s 134ms/step - loss: 4.2688


Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 40

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))